# Trading Strategy Evaluation Report

This notebook provides a basic evaluation report for the trading strategy, leveraging metrics calculated by `eval/metrics.py` and experiment tracking via MLflow. It visualizes the equity curve and displays key performance indicators.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
from mlflow.entities import ViewType
import os

# Set MLflow tracking URI (if not already set via environment variable)
# mlflow.set_tracking_uri("http://localhost:5000") # Uncomment if you have a local MLflow server

# Find the latest successful MLflow run that logged an equity_curve.csv artifact
runs = mlflow.search_runs(
    order_by=["start_time DESC"],
    max_results=1,
    filter_string="metrics.total_return IS NOT NULL AND attributes.status = 'FINISHED'"
)

if not runs.empty:
    latest_run_id = runs.iloc[0].run_id
    print(f"Found latest MLflow run: {latest_run_id}")

    # Download the equity curve artifact
    try:
        client = mlflow.tracking.MlflowClient()
        client.download_artifacts(run_id=latest_run_id, path="equity_curve.csv", dst_path=".")
        equity_curve_path = "equity_curve.csv"
        
        if os.path.exists(equity_curve_path):
            df_equity_curve = pd.read_csv(equity_curve_path)
            df_equity_curve['Date'] = pd.to_datetime(df_equity_curve['Date'])
            print("Equity curve loaded successfully:")
            print(df_equity_curve.head())
        else:
            df_equity_curve = pd.DataFrame()
            print(f"Equity curve artifact not found at {equity_curve_path}")

    except Exception as e:
        df_equity_curve = pd.DataFrame()
        print(f"Error downloading or loading equity curve artifact: {e}")
else:
    df_equity_curve = pd.DataFrame()
    print("No MLflow runs found with logged metrics and status 'FINISHED'.")

# Display logged metrics (if a run was found)
if not runs.empty:
    print("\nLogged Metrics:")
    metrics = runs.iloc[0][['metrics.total_return', 'metrics.sharpe_ratio', 'metrics.max_drawdown', 'metrics.annualized_volatility']].to_dict()
    for k, v in metrics.items():
        print(f"  {k.replace('metrics.', '').replace('_', ' ').title()}: {v:.4f}")


In [ ]:
if not df_equity_curve.empty:
    plt.figure(figsize=(12, 6))
    plt.plot(df_equity_curve['Date'], df_equity_curve['PortfolioValue'], label='Portfolio Value')
    plt.title('Equity Curve')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.grid(True)
    plt.legend()
    plt.show()
else:
    print("Cannot plot equity curve: DataFrame is empty.")
